In [1]:
#This code extracts GSP Zone data for specified Elexon run (II, SF, R1, R2, R3, RF or DF) for a particular time period.
import pandas as pd
import os
from glob import glob

# === CONFIGURATION ===
input_folder = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data\GSP CDCA-I029\AGV Data"
output_folder = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data"

# Date range (as datetime)
start_date = pd.to_datetime("2024-07-01")
end_date   = pd.to_datetime("2025-06-30")

# Expected GSP Group Ids
gsp_ids = ['_A','_B','_C','_D','_E','_F','_G','_H','_J','_K','_L','_M','_N','_P']

# Collect filtered data
all_data = []

for file in glob(os.path.join(input_folder, "*.csv")):
    try:
        df = pd.read_csv(file, dtype=str)
    except Exception as e:
        print(f"Skipping file {file}: {e}")
        continue

    df = df[df['Settlement Run Type'] == 'SF']
    df = df[['Settlement Date', 'Settlement Period', 'GSP Group Id', 'Import/Export Indicator', 'GSP Group Take Volume']]

    # Convert Settlement Date from yyyymmdd to datetime
    df['Settlement Date'] = pd.to_datetime(df['Settlement Date'], format='%Y%m%d', errors='coerce')

    # Drop rows with invalid dates
    df = df.dropna(subset=['Settlement Date'])

    # Filter by date range
    df = df[(df['Settlement Date'] >= start_date) & (df['Settlement Date'] <= end_date)]

    if df.empty:
        continue

    df['Settlement Period'] = df['Settlement Period'].astype(int)
    df['GSP Group Take Volume'] = pd.to_numeric(df['GSP Group Take Volume'], errors='coerce')

    df['GSP Group Take Volume'] = df.apply(
        lambda row: row['GSP Group Take Volume'] if row['Import/Export Indicator'] == 'I'
        else -row['GSP Group Take Volume'], axis=1
    )

    all_data.append(df)

# Combine and output
if not all_data:
    print("No data found in the specified date range.")
else:
    combined_df = pd.concat(all_data, ignore_index=True)

    # Pivot to wide format
    pivot_df = combined_df.pivot_table(
        index=['Settlement Date', 'Settlement Period'],
        columns='GSP Group Id',
        values='GSP Group Take Volume',
        aggfunc='sum'
    )

    pivot_df = pivot_df.reindex(columns=gsp_ids)
    pivot_df.reset_index(inplace=True)

    # Format for filename
    filename_start = start_date.strftime("%Y-%m-%d")
    filename_end = end_date.strftime("%Y-%m-%d")
    output_filename = f"GSP_Take_SF_{filename_start}_to_{filename_end}.csv"

    output_path = os.path.join(output_folder, output_filename)
    pivot_df.to_csv(output_path, index=False)

    print(f"Filtered output saved to: {output_path}")



Filtered output saved to: C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data\GSP_Take_SF_2024-07-01_to_2025-06-30.csv
